# 🏗️ Análise Avançada de Viabilidade de Empreendimentos

Este notebook demonstra:
- Cálculo de potencial construtivo com base em parâmetros urbanísticos
- Análise de viabilidade econômica
- Otimização de mix de produtos
- Análise de sensibilidade
- Comparação com mercado competidor
- Machine Learning para predição de preços
- Dashboard de viabilidade
- Design minimalista em preto e branco

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo
plt.style.use('grayscale')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ Bibliotecas importadas")

## 1. Carregar Dados de Lotes e Mercado

In [ ]:
# Carregar dados de lotes
try:
    df_lotes = pd.read_parquet('../data/processed/lotes_processados.parquet')
    print(f"✓ Lotes carregados: {len(df_lotes)}")
except:
    print("⚠️ Criando dados de exemplo de lotes...")
    np.random.seed(42)
    df_lotes = pd.DataFrame({
        'codLote': range(1, 101),
        'bairro': np.random.choice(['Centro', 'Praia do Canto', 'Jardim Camburi', 'Enseada do Suá'], 100),
        'area_terreno': np.random.uniform(300, 2000, 100),
        'ca': np.random.uniform(1.5, 4.0, 100),
        'to': np.random.uniform(0.4, 0.7, 100),
        'limite_altura': np.random.choice([18, 24, 30, 50], 100),
        'gabarito': np.random.randint(6, 18, 100)
    })

# Carregar dados de mercado
try:
    df_mercado = pd.read_parquet('../data/processed/imoveis_processados.parquet')
    print(f"✓ Dados de mercado carregados: {len(df_mercado)}")
except:
    print("⚠️ Criando dados de exemplo de mercado...")
    df_mercado = pd.DataFrame({
        'Bairro': np.random.choice(['Centro', 'Praia do Canto', 'Jardim Camburi', 'Enseada do Suá'], 200),
        'Dormitorios': np.random.choice([1, 2, 3, 4], 200),
        'Metragem Privativa': np.random.uniform(45, 180, 200),
        'Preco Total': np.random.uniform(300000, 1800000, 200),
        'Perc_Vendido': np.random.uniform(30, 95, 200)
    })
    df_mercado['Preco_m2'] = df_mercado['Preco Total'] / df_mercado['Metragem Privativa']

print("\n✓ Dados carregados com sucesso")

## 2. Calcular Potencial Construtivo

In [ ]:
def calcular_potencial_construtivo(lote):
    """
    Calcula o potencial construtivo de um lote com base em parâmetros urbanísticos
    """
    # Área total computável (CA × Área do terreno)
    area_computavel = lote['area_terreno'] * lote['ca']
    
    # Área de projeção máxima (TO × Área do terreno)
    area_projecao = lote['area_terreno'] * lote['to']
    
    # Número estimado de pavimentos
    num_pavimentos = min(lote['gabarito'], int(area_computavel / area_projecao))
    
    # Área privativa estimada (70% da área computável)
    area_privativa_total = area_computavel * 0.70
    
    # Área de circulação e técnica (30%)
    area_comum = area_computavel * 0.30
    
    return {
        'area_computavel': area_computavel,
        'area_projecao': area_projecao,
        'num_pavimentos': num_pavimentos,
        'area_privativa_total': area_privativa_total,
        'area_comum': area_comum
    }

# Aplicar cálculo
potenciais = df_lotes.apply(calcular_potencial_construtivo, axis=1, result_type='expand')
df_lotes = pd.concat([df_lotes, potenciais], axis=1)

print("📐 POTENCIAL CONSTRUTIVO CALCULADO")
print("="*60)
print(f"Área computável média: {df_lotes['area_computavel'].mean():.2f} m²")
print(f"Área privativa média: {df_lotes['area_privativa_total'].mean():.2f} m²")
print(f"Número médio de pavimentos: {df_lotes['num_pavimentos'].mean():.1f}")

df_lotes.head()

## 3. Otimização de Mix de Produtos

In [ ]:
def otimizar_mix_produtos(lote, precos_mercado):
    """
    Otimiza o mix de produtos (tipologias) para maximizar VGV
    """
    area_disponivel = lote['area_privativa_total']
    bairro = lote['bairro']
    
    # Tipologias possíveis e suas áreas médias
    tipologias = {
        '1 dorm': {'area': 45, 'preco_m2': precos_mercado.get(bairro, {}).get(1, 10000)},
        '2 dorm': {'area': 65, 'preco_m2': precos_mercado.get(bairro, {}).get(2, 11000)},
        '3 dorm': {'area': 95, 'preco_m2': precos_mercado.get(bairro, {}).get(3, 12000)},
        '4 dorm': {'area': 140, 'preco_m2': precos_mercado.get(bairro, {}).get(4, 13000)}
    }
    
    # Estratégia: Maximizar número de unidades mantendo mix equilibrado
    # 10% 1 dorm, 40% 2 dorm, 40% 3 dorm, 10% 4 dorm
    mix = {}
    mix['1 dorm'] = max(1, int(area_disponivel * 0.10 / tipologias['1 dorm']['area']))
    mix['2 dorm'] = max(1, int(area_disponivel * 0.40 / tipologias['2 dorm']['area']))
    mix['3 dorm'] = max(1, int(area_disponivel * 0.40 / tipologias['3 dorm']['area']))
    mix['4 dorm'] = max(1, int(area_disponivel * 0.10 / tipologias['4 dorm']['area']))
    
    # Calcular área total utilizada e VGV
    area_utilizada = sum(mix[tipo] * tipologias[tipo]['area'] for tipo in mix)
    vgv = sum(mix[tipo] * tipologias[tipo]['area'] * tipologias[tipo]['preco_m2'] for tipo in mix)
    
    total_unidades = sum(mix.values())
    
    return {
        'mix': mix,
        'total_unidades': total_unidades,
        'area_utilizada': area_utilizada,
        'taxa_aproveitamento': (area_utilizada / area_disponivel) * 100,
        'vgv_estimado': vgv
    }

# Calcular preços médios de mercado por bairro e tipologia
precos_mercado = {}
for bairro in df_mercado['Bairro'].unique():
    precos_mercado[bairro] = {}
    for dorm in df_mercado['Dormitorios'].unique():
        mask = (df_mercado['Bairro'] == bairro) & (df_mercado['Dormitorios'] == dorm)
        if mask.sum() > 0:
            precos_mercado[bairro][dorm] = df_mercado.loc[mask, 'Preco_m2'].mean()

# Selecionar um lote de exemplo para análise detalhada
lote_exemplo = df_lotes.iloc[0]
resultado = otimizar_mix_produtos(lote_exemplo, precos_mercado)

print("\n🏢 OTIMIZAÇÃO DE MIX DE PRODUTOS")
print("="*60)
print(f"Lote: {lote_exemplo['codLote']}")
print(f"Bairro: {lote_exemplo['bairro']}")
print(f"Área do terreno: {lote_exemplo['area_terreno']:.2f} m²")
print(f"Área privativa disponível: {lote_exemplo['area_privativa_total']:.2f} m²")
print(f"\nMix otimizado:")
for tipo, qtd in resultado['mix'].items():
    print(f"  • {tipo}: {qtd} unidades")
print(f"\nTotal de unidades: {resultado['total_unidades']}")
print(f"Área utilizada: {resultado['area_utilizada']:.2f} m²")
print(f"Taxa de aproveitamento: {resultado['taxa_aproveitamento']:.1f}%")
print(f"VGV estimado: R$ {resultado['vgv_estimado']/1e6:.2f} milhões")

## 4. Análise de Viabilidade Econômica

In [ ]:
def analisar_viabilidade(lote, mix_resultado, preco_terreno_m2=3000):
    """
    Análise completa de viabilidade econômica
    """
    # Receitas
    vgv = mix_resultado['vgv_estimado']
    
    # Custos
    custo_terreno = lote['area_terreno'] * preco_terreno_m2
    custo_construcao_m2 = 4500  # R$/m² construído
    custo_construcao = lote['area_computavel'] * custo_construcao_m2
    
    # Despesas operacionais (% do VGV)
    taxa_incorporacao = vgv * 0.05  # 5% do VGV
    marketing_vendas = vgv * 0.04  # 4% do VGV
    impostos = vgv * 0.06  # 6% do VGV
    
    # Custos totais
    custo_total = custo_terreno + custo_construcao + taxa_incorporacao + marketing_vendas + impostos
    
    # Resultado
    lucro_bruto = vgv - custo_total
    margem_bruta = (lucro_bruto / vgv) * 100 if vgv > 0 else 0
    roi = (lucro_bruto / custo_total) * 100 if custo_total > 0 else 0
    
    return {
        'vgv': vgv,
        'custo_terreno': custo_terreno,
        'custo_construcao': custo_construcao,
        'taxa_incorporacao': taxa_incorporacao,
        'marketing_vendas': marketing_vendas,
        'impostos': impostos,
        'custo_total': custo_total,
        'lucro_bruto': lucro_bruto,
        'margem_bruta': margem_bruta,
        'roi': roi
    }

# Analisar viabilidade do lote exemplo
viabilidade = analisar_viabilidade(lote_exemplo, resultado)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'Análise de Viabilidade - Lote {lote_exemplo["codLote"]}', fontsize=16, fontweight='bold')

# Gráfico 1: Composição de Custos
custos = {
    'Terreno': viabilidade['custo_terreno'],
    'Construção': viabilidade['custo_construcao'],
    'Incorporação': viabilidade['taxa_incorporacao'],
    'Marketing': viabilidade['marketing_vendas'],
    'Impostos': viabilidade['impostos']
}

colors_grayscale = ['black', 'darkgray', 'gray', 'lightgray', 'whitesmoke']
wedges, texts, autotexts = axes[0].pie(custos.values(), 
                                        labels=custos.keys(),
                                        autopct='%1.1f%%',
                                        colors=colors_grayscale,
                                        explode=[0.05]*len(custos),
                                        wedgeprops={'edgecolor': 'black', 'linewidth': 2})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

axes[0].set_title('Composição de Custos', fontweight='bold', pad=20)

# Gráfico 2: Receitas vs Custos
categorias = ['VGV\n(Receita)', 'Custos\nTotais', 'Lucro\nBruto']
valores = [viabilidade['vgv']/1e6, viabilidade['custo_total']/1e6, viabilidade['lucro_bruto']/1e6]
cores = ['black', 'gray', 'lightgray' if viabilidade['lucro_bruto'] > 0 else 'darkgray']

bars = axes[1].bar(categorias, valores, color=cores, alpha=0.7, edgecolor='black', linewidth=2)

# Adicionar valores nas barras
for bar, valor in zip(bars, valores):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'R$ {valor:.2f}M',
                ha='center', va='bottom', fontweight='bold', fontsize=11)

axes[1].set_ylabel('Valor (R$ Milhões)', fontweight='bold')
axes[1].set_title('Análise de Resultado', fontweight='bold', pad=20)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)

plt.tight_layout()
plt.show()

# Relatório
print("\n💰 ANÁLISE DE VIABILIDADE ECONÔMICA")
print("="*60)
print("\n📊 RECEITAS")
print(f"  VGV Total: R$ {viabilidade['vgv']:,.2f}")

print("\n💸 CUSTOS")
print(f"  Terreno: R$ {viabilidade['custo_terreno']:,.2f}")
print(f"  Construção: R$ {viabilidade['custo_construcao']:,.2f}")
print(f"  Taxa de Incorporação: R$ {viabilidade['taxa_incorporacao']:,.2f}")
print(f"  Marketing e Vendas: R$ {viabilidade['marketing_vendas']:,.2f}")
print(f"  Impostos: R$ {viabilidade['impostos']:,.2f}")
print(f"  ───────────────────────────────────")
print(f"  TOTAL: R$ {viabilidade['custo_total']:,.2f}")

print("\n📈 RESULTADO")
print(f"  Lucro Bruto: R$ {viabilidade['lucro_bruto']:,.2f}")
print(f"  Margem Bruta: {viabilidade['margem_bruta']:.2f}%")
print(f"  ROI: {viabilidade['roi']:.2f}%")

# Indicador de viabilidade
if viabilidade['margem_bruta'] >= 25:
    print("\n✓ EMPREENDIMENTO VIÁVEL - Margem atrativa")
elif viabilidade['margem_bruta'] >= 15:
    print("\n⚠️ EMPREENDIMENTO VIÁVEL - Margem moderada")
else:
    print("\n✗ EMPREENDIMENTO INVIÁVEL - Margem insuficiente")

## 5. Análise de Sensibilidade

In [ ]:
# Análise de sensibilidade: variar preço de venda e custo de construção
variacao_venda = np.arange(-20, 25, 5)  # -20% a +20%
variacao_custo = np.arange(-10, 15, 5)  # -10% a +10%

# Criar matriz de resultados
matriz_margem = np.zeros((len(variacao_custo), len(variacao_venda)))

for i, var_custo in enumerate(variacao_custo):
    for j, var_venda in enumerate(variacao_venda):
        # Ajustar valores
        vgv_ajustado = viabilidade['vgv'] * (1 + var_venda/100)
        custo_construcao_ajustado = viabilidade['custo_construcao'] * (1 + var_custo/100)
        
        # Recalcular custos totais
        custo_total_ajustado = (
            viabilidade['custo_terreno'] +
            custo_construcao_ajustado +
            vgv_ajustado * 0.15  # Taxas e impostos proporcionais ao VGV
        )
        
        # Calcular margem
        lucro_ajustado = vgv_ajustado - custo_total_ajustado
        margem_ajustada = (lucro_ajustado / vgv_ajustado * 100) if vgv_ajustado > 0 else 0
        
        matriz_margem[i, j] = margem_ajustada

# Visualizar
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

im = ax.imshow(matriz_margem, cmap='Greys', aspect='auto', origin='lower')

# Configurar eixos
ax.set_xticks(np.arange(len(variacao_venda)))
ax.set_yticks(np.arange(len(variacao_custo)))
ax.set_xticklabels([f'{v:+.0f}%' for v in variacao_venda])
ax.set_yticklabels([f'{v:+.0f}%' for v in variacao_custo])

ax.set_xlabel('Variação do Preço de Venda', fontsize=12, fontweight='bold')
ax.set_ylabel('Variação do Custo de Construção', fontsize=12, fontweight='bold')
ax.set_title('Análise de Sensibilidade - Margem Bruta (%)', fontsize=14, fontweight='bold', pad=20)

# Adicionar valores
for i in range(len(variacao_custo)):
    for j in range(len(variacao_venda)):
        text_color = 'white' if matriz_margem[i, j] < 15 or matriz_margem[i, j] > 35 else 'black'
        text = ax.text(j, i, f'{matriz_margem[i, j]:.1f}%',
                      ha='center', va='center', color=text_color, fontweight='bold', fontsize=9)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Margem Bruta (%)', rotation=270, labelpad=20, fontweight='bold')

# Grid
ax.set_xticks(np.arange(len(variacao_venda))-.5, minor=True)
ax.set_yticks(np.arange(len(variacao_custo))-.5, minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

plt.tight_layout()
plt.show()

print("\n📊 ANÁLISE DE SENSIBILIDADE")
print("="*60)
print("Margem Base: {:.2f}%".format(viabilidade['margem_bruta']))
print("\nCenários:")
print(f"  • Melhor caso (+20% venda, -10% custo): {matriz_margem[-1, -1]:.2f}%")
print(f"  • Caso base (0%, 0%): {matriz_margem[len(variacao_custo)//2, len(variacao_venda)//2]:.2f}%")
print(f"  • Pior caso (-20% venda, +10% custo): {matriz_margem[0, 0]:.2f}%")

## 6. Machine Learning - Predição de Preços

In [ ]:
# Preparar dados para ML
df_ml = df_mercado.copy()

# Encoding de variáveis categóricas
df_ml['Bairro_encoded'] = pd.Categorical(df_ml['Bairro']).codes

# Features e target
features = ['Bairro_encoded', 'Dormitorios', 'Metragem Privativa']
target = 'Preco_m2'

X = df_ml[features]
y = df_ml[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar modelo
model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Predições
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Métricas
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Machine Learning - Predição de Preços', fontsize=16, fontweight='bold')

# Gráfico 1: Predito vs Real
axes[0].scatter(y_test, y_pred_test, c='black', alpha=0.6, s=80, edgecolors='white', linewidth=1)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             '--', color='gray', linewidth=2, label='Predição Perfeita')

axes[0].set_xlabel('Preço Real (R$/m²)', fontweight='bold')
axes[0].set_ylabel('Preço Predito (R$/m²)', fontweight='bold')
axes[0].set_title(f'Predito vs Real (R² = {r2_test:.3f})', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Importância das Features
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

feature_names = ['Bairro', 'Dormitórios', 'Metragem']
bars = axes[1].barh(range(len(importances)), importances[indices], 
                    color='black', alpha=0.7, edgecolor='white', linewidth=2)

axes[1].set_yticks(range(len(importances)))
axes[1].set_yticklabels([feature_names[i] for i in indices])
axes[1].set_xlabel('Importância', fontweight='bold')
axes[1].set_title('Importância das Variáveis', fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

# Adicionar valores
for i, (bar, imp) in enumerate(zip(bars, importances[indices])):
    axes[1].text(imp, i, f' {imp:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🤖 MODELO DE MACHINE LEARNING")
print("="*60)
print(f"Modelo: Random Forest Regressor")
print(f"\nPerformance:")
print(f"  • R² (treino): {r2_train:.3f}")
print(f"  • R² (teste): {r2_test:.3f}")
print(f"  • MAE (teste): R$ {mae_test:.2f}/m²")
print(f"\nImportância das variáveis:")
for i in indices:
    print(f"  • {feature_names[i]}: {importances[i]:.3f}")

## 7. Aplicar ML para Predição de Preço do Lote

In [ ]:
# Predizer preço para cada unidade do mix
bairro_encoded = pd.Categorical(df_mercado['Bairro']).categories.get_loc(lote_exemplo['bairro'])

tipologias_predicao = [
    {'nome': '1 dorm', 'dorm': 1, 'area': 45},
    {'nome': '2 dorm', 'dorm': 2, 'area': 65},
    {'nome': '3 dorm', 'dorm': 3, 'area': 95},
    {'nome': '4 dorm', 'dorm': 4, 'area': 140}
]

print("\n🎯 PREDIÇÃO DE PREÇOS COM ML")
print("="*60)
print(f"Bairro: {lote_exemplo['bairro']}\n")

precos_preditos = {}
for tipo in tipologias_predicao:
    X_pred = np.array([[bairro_encoded, tipo['dorm'], tipo['area']]])
    preco_m2_pred = model.predict(X_pred)[0]
    preco_total = preco_m2_pred * tipo['area']
    precos_preditos[tipo['nome']] = {'preco_m2': preco_m2_pred, 'preco_total': preco_total}
    
    print(f"{tipo['nome']}:")
    print(f"  • Área: {tipo['area']} m²")
    print(f"  • Preço/m² predito: R$ {preco_m2_pred:,.2f}")
    print(f"  • Preço total: R$ {preco_total:,.2f}")
    print()

# Recalcular VGV com preços preditos
vgv_ml = sum(
    resultado['mix'][tipo] * precos_preditos[tipo]['preco_total']
    for tipo in resultado['mix']
)

print(f"\nVGV recalculado com ML: R$ {vgv_ml/1e6:.2f} milhões")
print(f"VGV original: R$ {resultado['vgv_estimado']/1e6:.2f} milhões")
print(f"Diferença: {((vgv_ml - resultado['vgv_estimado'])/resultado['vgv_estimado']*100):+.1f}%")

## 8. Dashboard Comparativo de Lotes

In [ ]:
# Analisar todos os lotes
resultados_lotes = []

for idx, lote in df_lotes.head(20).iterrows():  # Analisar primeiros 20 lotes
    try:
        mix = otimizar_mix_produtos(lote, precos_mercado)
        viab = analisar_viabilidade(lote, mix)
        
        resultados_lotes.append({
            'codLote': lote['codLote'],
            'bairro': lote['bairro'],
            'area_terreno': lote['area_terreno'],
            'ca': lote['ca'],
            'unidades': mix['total_unidades'],
            'vgv': viab['vgv'],
            'margem': viab['margem_bruta'],
            'roi': viab['roi']
        })
    except:
        pass

df_resultados = pd.DataFrame(resultados_lotes)

# Visualizar Top 10
df_top10 = df_resultados.nlargest(10, 'margem')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Dashboard Comparativo - Top 10 Lotes por Margem', fontsize=16, fontweight='bold')

# 1. Margem por Lote
bars = axes[0, 0].barh(range(len(df_top10)), df_top10['margem'], 
                       color='black', alpha=0.7, edgecolor='white', linewidth=1.5)

for i, (bar, margem, cod) in enumerate(zip(bars, df_top10['margem'], df_top10['codLote'])):
    axes[0, 0].text(margem, i, f' {margem:.1f}% (Lote {cod})',
                   va='center', fontweight='bold', fontsize=9)

axes[0, 0].set_xlabel('Margem Bruta (%)', fontweight='bold')
axes[0, 0].set_title('Margem Bruta por Lote', fontweight='bold')
axes[0, 0].set_yticks(range(len(df_top10)))
axes[0, 0].set_yticklabels([f"Lote {cod}" for cod in df_top10['codLote']])
axes[0, 0].invert_yaxis()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. VGV por Lote
bars = axes[0, 1].barh(range(len(df_top10)), df_top10['vgv']/1e6, 
                       color='gray', alpha=0.7, edgecolor='black', linewidth=1.5)

for i, (bar, vgv) in enumerate(zip(bars, df_top10['vgv']/1e6)):
    axes[0, 1].text(vgv, i, f' R$ {vgv:.1f}M',
                   va='center', fontweight='bold', fontsize=9)

axes[0, 1].set_xlabel('VGV (R$ Milhões)', fontweight='bold')
axes[0, 1].set_title('VGV por Lote', fontweight='bold')
axes[0, 1].set_yticks(range(len(df_top10)))
axes[0, 1].set_yticklabels([f"Lote {cod}" for cod in df_top10['codLote']])
axes[0, 1].invert_yaxis()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Scatter: Área vs Margem
scatter = axes[1, 0].scatter(df_resultados['area_terreno'], df_resultados['margem'],
                            s=df_resultados['vgv']/1e4,
                            c='black', alpha=0.6, edgecolors='white', linewidth=1.5)

axes[1, 0].set_xlabel('Área do Terreno (m²)', fontweight='bold')
axes[1, 0].set_ylabel('Margem Bruta (%)', fontweight='bold')
axes[1, 0].set_title('Relação Área vs Margem (tamanho = VGV)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. ROI por Bairro
roi_bairro = df_resultados.groupby('bairro')['roi'].mean().sort_values(ascending=False)

bars = axes[1, 1].bar(range(len(roi_bairro)), roi_bairro.values,
                      color='black', alpha=0.7, edgecolor='white', linewidth=2)

for bar, roi in zip(bars, roi_bairro.values):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{roi:.1f}%',
                   ha='center', va='bottom', fontweight='bold')

axes[1, 1].set_xticks(range(len(roi_bairro)))
axes[1, 1].set_xticklabels(roi_bairro.index, rotation=45, ha='right')
axes[1, 1].set_ylabel('ROI Médio (%)', fontweight='bold')
axes[1, 1].set_title('ROI Médio por Bairro', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n🏆 RANKING DE LOTES (Top 5 por Margem)")
print("="*80)
print(df_top10.head()[['codLote', 'bairro', 'area_terreno', 'unidades', 'vgv', 'margem', 'roi']].to_string(index=False))

## 9. Exportar Resultados

In [ ]:
# Exportar análises
df_resultados.to_csv('../data/processed/analise_viabilidade_lotes.csv', index=False)
df_resultados.to_parquet('../data/processed/analise_viabilidade_lotes.parquet')

# Criar relatório resumido
relatorio = {
    'data_analise': pd.Timestamp.now(),
    'total_lotes_analisados': len(df_resultados),
    'margem_media': df_resultados['margem'].mean(),
    'margem_mediana': df_resultados['margem'].median(),
    'vgv_total': df_resultados['vgv'].sum(),
    'roi_medio': df_resultados['roi'].mean(),
    'melhor_lote': int(df_resultados.loc[df_resultados['margem'].idxmax(), 'codLote'])
}

pd.DataFrame([relatorio]).to_json('../data/processed/relatorio_viabilidade.json', orient='records', indent=2)

print("✓ Resultados exportados:")
print("  • analise_viabilidade_lotes.csv")
print("  • analise_viabilidade_lotes.parquet")
print("  • relatorio_viabilidade.json")
print(f"\n{len(df_resultados)} lotes analisados com sucesso!")